In [2]:
import sys
sys.path.insert(0, '../')

import flammkuchen as fl
import tensorflow as tf
from tensorflow.keras.models import load_model
from utils.params import Params
from neural_networks.src.dataloader import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import librosa
import efficientnet.tfkeras as efn
import scipy

In [3]:
#### Change 'PATH_GA_O2' to the path where your trained GA model is located ####
PATH_GA_O2 = "../neural_networks/GeneticAlgorithm/Results_GA/GA_20211115-105541_objective2/best_models_per_generation/gen17_Fitness2.4656.hdf5"

PATH_PARAMS = "../neural_networks/params.json"

In [4]:
# load parameter
params = Params(PATH_PARAMS)
params.signal_type = "NSA"

# load test data
data_loader = DataLoader(params=params, nb_classes=4)
X_test, Y_test = data_loader.get_test_data()


Currently loading data: P6_1


  0%|          | 0/1882 [00:00<?, ?it/s]


Currently loading data: P6_2


  0%|          | 0/1790 [00:00<?, ?it/s]


Currently loading data: P6_3


  0%|          | 0/1805 [00:00<?, ?it/s]

In [5]:
model_ga_o2 = load_model(PATH_GA_O2)

In [6]:
def class_activation_map(model, image):
    gap_weights = model.layers[-1].get_weights()[0]

    cam_model = tf.keras.models.Model(inputs=model.input, outputs=(model.layers[-3].output, model.layers[-1].output))
    features, results = cam_model.predict(image[None,..., None])

    # bilinear upsampling
    cam_features = scipy.ndimage.zoom(features[0, ...], (64/8, 64/8, 1), order=2)

    pred = np.argmax(results[0])
    cam_weights = gap_weights[:, pred]
    cam_output = np.dot(cam_features, cam_weights)
    
    return pred, cam_output

In [7]:
def get_cams(model, X_test):
    """
    Crates the Class Activation Maps of each sample in X_test.
    @param model: Model to use for extracting CAMs.
    @param true_label: If true, the CAMs of the true labels are considered. If False, the CAMs of the predicted events are returned.
    :return: Four lists for each event type containing all pairs of [input_image, output_CAM].
    """
    cam_lists = [list(), list(), list(), list()]  # cough, dryswallow, throatclear, no event
    
    for x in tqdm(X_test):
        pred, cam_output = class_activation_map(model, x)
        cam_lists[pred].append([x, cam_output])

    return cam_lists[0], cam_lists[1], cam_lists[2], cam_lists[3]

In [8]:
coughs, dryswallows, throatclears, noevents = get_cams(model_ga_o2, X_test)

  0%|          | 0/5462 [00:00<?, ?it/s]

# Create dict with CAM examples for each airway symptom and save it

In [32]:
d = dict()
d

{}

In [33]:
idx = 2
d = dict(d, cough=coughs[idx][0])
d = dict(d, cough_cam=coughs[idx][1])

In [34]:
c_sum = np.zeros((64, 64))
for cough in coughs:
    c_sum += cough[1]
    
d = dict(d, cough_averaged=c_sum)

### Dry swallow

In [35]:
idx = 11
d = dict(d, dryswallow=dryswallows[idx][0])
d = dict(d, dryswallow_cam=dryswallows[idx][1])

In [36]:
ds_sum = np.zeros((64, 64))
for ds in dryswallows:
    ds_sum += ds[1]

d = dict(d, dryswallow_averaged=ds_sum)

### Throat clear

In [37]:
idx = 4
d = dict(d, throatclear=throatclears[idx][0])
d = dict(d, throatclear_cam=throatclears[idx][1])

In [38]:
tc_sum = np.zeros((64, 64))
for tc in throatclears:
    tc_sum += tc[1]

d = dict(d, throatclear_averaged=tc_sum)

### No event

In [39]:
idx = 208
d = dict(d, noevent=noevents[idx][0])
d = dict(d, noevent_cam=noevents[idx][1])

In [40]:
ne_sum = np.zeros((64, 64))
for ne in noevents:
    ne_sum += ne[1]

d = dict(d, noevent_averaged=ne_sum)

In [41]:
fl.save("results_CAMs.vfp", d)